In [ ]:
import pandas as pd
import re

#nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')

#sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df_train1 = pd.read_csv('/content/test_Arabic_tweets_negative_20190413.tsv', sep='\t',header=None,encoding='utf-8',index_col=False)
df_train2 = pd.read_csv('/content/test_Arabic_tweets_positive_20190413.tsv', sep='\t',header=None,encoding='utf-8',index_col=False)
df_test1 = pd.read_csv('/content/test_Arabic_tweets_negative_20190413.tsv', sep='\t',header=None,encoding='utf-8',index_col=False)
df_test2 = pd.read_csv('/content/test_Arabic_tweets_positive_20190413.tsv', sep='\t',header=None,encoding='utf-8',index_col=False)


In [ ]:
df_train2.head()

,0,1
0,pos,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...
1,pos,صباحك خيرات ومسرات 🌸
2,pos,#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫...
3,pos,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويت...
4,pos,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع ا...


In [ ]:
df_train =pd.concat([df_train1,df_train2],axis=0)
df_train.columns=["y","x"]

In [ ]:
df_test =pd.concat([df_test1,df_test2],axis=0)
df_test.columns=["y","x"]

In [ ]:
df_train.head()

,y,x
0,neg,حتى الايتونز خربتوه مو صاحين انتو؟؟ 😭
1,neg,واحد تبع النظام السوري يقول أن المخابرات السور...
2,neg,الى متى التعامل السئ للخادمات وعدم احترامهم وك...
3,neg,رايح جاي ي طحلبي 🐸 #الهلال_الاهلي
4,neg,تتمغط ومعها سداع 😫


In [ ]:
df_test.head()

,y,x
0,neg,حتى الايتونز خربتوه مو صاحين انتو؟؟ 😭
1,neg,واحد تبع النظام السوري يقول أن المخابرات السور...
2,neg,الى متى التعامل السئ للخادمات وعدم احترامهم وك...
3,neg,رايح جاي ي طحلبي 🐸 #الهلال_الاهلي
4,neg,تتمغط ومعها سداع 😫


In [ ]:
df_train_shuffled = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
df_test_shuffled = df_test.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
df_train_shuffled.head()

,y,x
0,neg,/4/12 - الاتحاد يتوج بطلا للدوري. /4/12 - الات...
1,pos,آمين يارب 🌹
2,neg,بيجون للسعودية ؟ 🤔 ولا يفكرون يجون ؟ 🤔
3,pos,#مانشستر_يونايتد_برشلونه #صباح_الخميس #الاهلي_...
4,pos,💡 هاشتاق أداء بلاك بينك في كوتشيلا قي المركز ا...


In [ ]:
len(df_train_shuffled)

11520

In [ ]:
len(df_test_shuffled)

11520

In [ ]:
df_test_shuffled.isna().sum()

,0
y,0
x,0


In [ ]:
df_train_shuffled.isna().sum()

,0
y,0
x,0


In [ ]:
df_train_shuffled.duplicated().sum()

2679

In [ ]:
df_train_shuffled.drop_duplicates(inplace=True)

In [ ]:
df_test_shuffled.duplicated().sum()

2679

In [ ]:
df_test_shuffled.drop_duplicates(inplace=True)

In [ ]:
len(df_test_shuffled)

8841

In [ ]:
names =["_","السبت", "الأحد", "الإثنين", "الثلاثاء",
    "الأربعاء", "الخميس", "الجمعة", "يناير", "فبراير", "مارس", "إبريل", "مايو",
    "يونيو", "يوليو", "أغسطس", "سبتمبر",
    "أكتوبر", "نوفمبر", "ديسمبر"]
# Arabic stopwords list
arabic_stopwords = set(stopwords.words('arabic'))

# Initialize the Arabic stemmer
stemmer = SnowballStemmer("arabic")

# Function to clean and preprocess text
def clean_text(text):
    pattern ="|".join(names)
    text = re.sub('[إأآا]', 'ا', text)
    text = re.sub('ؤ', 'ء', text)
    text = re.sub('ئ', 'ء', text)
    text = re.sub('ة', 'ه', text)
    text = re.sub('ى', 'ي', text)
    text = re.sub(pattern, '', text)
    text = re.sub(r'\/|\!|\$|\%|\^|\&|\*|\(|\)|\?|\.|\'|\"', '', text)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'\d+', '', text)



    # Tokenization
    tokens = word_tokenize(text)
    # Stopwords removal and stemming
    tokens = [stemmer.stem(word) for word in tokens if word not in arabic_stopwords]

    return ' '.join(tokens)




In [ ]:
# Apply the clean_text function to the dataset
df_train_shuffled['cleaned_text'] = df_train_shuffled['x'].apply(clean_text)

In [ ]:
df_test_shuffled['cleaned_text'] = df_test_shuffled['x'].apply(clean_text)

In [ ]:
df_test_shuffled.head()

,y,x,cleaned_text
0,neg,/4/12 - الاتحاد يتوج بطلا للدوري. /4/12 - الات...,اتحاد يتوج بطل دور اتحاد مهدد هبوط را دهر مخ...
1,pos,آمين يارب 🌹,ام يارب 🌹
2,neg,بيجون للسعودية ؟ 🤔 ولا يفكرون يجون ؟ 🤔,يجو سعوديه 🤔 يفكر يجو 🤔
3,pos,#مانشستر_يونايتد_برشلونه #صباح_الخميس #الاهلي_...,مانشستريونايتدبرشل اهليالهلال نصرالاتحاد اعلام...
4,pos,💡 هاشتاق أداء بلاك بينك في كوتشيلا قي المركز ا...,💡 هاشتاق اداء بلا بين وتشيل قي مركز ثان حول عا...


In [ ]:
df_train_shuffled.drop('x',axis =1,inplace=True)

In [ ]:
x_train,y_train,x_test,y_test = df_train_shuffled['cleaned_text'],df_train_shuffled['y'],df_test_shuffled['cleaned_text'],df_test_shuffled['y']

In [ ]:
x_test.shape

(8841,)

In [ ]:
x_train.shape

(8841,)

In [ ]:
x_train = pd.concat([x_train,x_test[:5000]],axis=0)
y_train = pd.concat([y_train,y_test[:5000]],axis=0)


In [ ]:
len(x_train)

13841

In [ ]:
x_test =x_test[5000:]
y_test = y_test[5000:]


In [ ]:
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [ ]:
x_train.tail()

,cleaned_text
13836,الله يرحمج 🌚
13837,مشاعرهالليلهتقول اصحاب مشاعر اي الم تتحدث 💔
13838,شنو جذب 🤔
13839,الخ الخ جمهور منبطح دايم اتحاد مسو علاق مادمر ...
13840,صباحيه قول 😂


In [ ]:


# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)

X_train_sequences = tokenizer.texts_to_sequences(x_train)
X_test_sequences = tokenizer.texts_to_sequences(x_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=100, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=100, padding='post', truncating='post')

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

# Convert labels to categorical (one-hot encoding)
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# LSTM Model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=128, input_length=100),  # Embedding layer
    tf.keras.layers.LSTM(128, return_sequences=True),  # LSTM layer
    tf.keras.layers.LSTM(64),  # Another LSTM layer
    tf.keras.layers.Dense(32, activation='relu'),  # Dense layer
    tf.keras.layers.Dropout(0.5),  # Dropout layer for regularization
    tf.keras.layers.Dense(2, activation='softmax')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_padded, y_train_categorical, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test_categorical)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


390/390 ━━━━━━━━━━━━━━━━━━━━ 106s 262ms/step - accuracy: 0.5003 - loss: 0.6940 - val_accuracy: 0.5119 - val_loss: 0.6929
Epoch 2/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 142s 263ms/step - accuracy: 0.5031 - loss: 0.6931 - val_accuracy: 0.5141 - val_loss: 0.6930
Epoch 3/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 141s 261ms/step - accuracy: 0.5044 - loss: 0.6934 - val_accuracy: 0.5119 - val_loss: 0.6930
Epoch 4/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 137s 248ms/step - accuracy: 0.5003 - loss: 0.6934 - val_accuracy: 0.5141 - val_loss: 0.6928
Epoch 5/5
390/390 ━━━━━━━━━━━━━━━━━━━━ 145s 255ms/step - accuracy: 0.5068 - loss: 0.6934 - val_accuracy: 0.5141 - val_loss: 0.6929


KeyboardInterrupt: 